<a href="https://colab.research.google.com/github/silverstar0727/pragmatism-AI/blob/main/VGG(cifar10).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q wandb

In [2]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras

from wandb.keras import WandbCallback
import wandb

wandb.login()

wandb: Currently logged in as: jeongmin (use `wandb login --relogin` to force relogin)


True

In [8]:
cifar10 = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=3)

In [17]:
cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M']
}


In [18]:
class VGG(tf.keras.Model):
    def __init__(self, model_code, hid_dim):
        super(VGG, self).__init__()
        self.hid_dim = hid_dim
        self.model_code = model_code

        self.ls = self._make_layers(self.model_code)
        self.flatten = tf.keras.layers.Flatten()
        self.linear = tf.keras.layers.Dense(self.hid_dim, activation='relu')
        self.outputs = tf.keras.layers.Dense(10, activation='softmax')
        
    def call(self, x):
        for layer in self.ls:
            x = layer(x)
        x = self.flatten(x)
        x = self.linear(x)
        x = self.outputs(x)
        return x
    
    def _make_layers(self, model_code):
        layers = [tf.keras.layers.Conv2D(input_shape=(32, 32,3),
                                            filters=64,
                                            kernel_size=3,
                                            padding="same", 
                                            activation="relu")]
        for x in cfg[model_code]:
            if x != 'M':
                layers.append(tf.keras.layers.Conv2D(filters=x, 
                                       kernel_size=3,
                                       activation='relu', 
                                       padding='same'))
            else:
                layers.append(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2))

        return layers

In [30]:
def experiment():
    config_defaults = {
        'model_code': 'VGG11',
        'epochs': 2,
        'batch_size': 1024,
        'hid_dim': 512,
        'lr': 0.1
    }

    wandb.init(project='VGGs', config=config_defaults, magic=True)
    args = wandb.config
    
    # Prepare dataloader
    trainloader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    trainloader = trainloader.shuffle(1024).batch(args.batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    valloader = tf.data.Dataset.from_tensor_slices((x_val, y_val))
    valloader = valloader.batch(args.batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    testloader = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    testloader = testloader.batch(args.batch_size).prefetch(tf.data.experimental.AUTOTUNE)

    # init
    keras.backend.clear_session()
    
    net = VGG(args.model_code, args.hid_dim)
    optimizer = tf.keras.optimizers.Adam(learning_rate=args.lr)
    net.compile(optimizer=optimizer,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    result = net.fit(
        trainloader, 
        epochs=args.epochs, 
        validation_data=valloader, 
        batch_size=args.batch_size,
        callbacks=[WandbCallback(save_model=False)])

    test_result = net.evaluate(testloader, callbacks=[WandbCallback(save_model=False)])

In [31]:
sweep_config = {
    'method': 'grid',
    'parameters': {
        'model_code': {
            'values': ['VGG11', 'VGG13', 'VGG16', 'VGG19']
        },
        'lr': {
            'values': [0.01, 0.001, 0.0001]
        },
        'epochs': {
            'values': [10]
        },
        'batch_size': {
            'values': [1024]
        },
        'hid_dim': {
            'values': [64, 1024]
        }
    }
}

import wandb
sweep_id = wandb.sweep(sweep_config)

wandb.agent(sweep_id, function=experiment)

Create sweep with ID: maudemkc
Sweep URL: https://wandb.ai/jeongmin/uncategorized/sweeps/maudemkc


wandb: Agent Starting Run: tjl8n4k1 with config:
wandb: 	batch_size: 1024
wandb: 	epochs: 10
wandb: 	hid_dim: 64
wandb: 	lr: 0.001
wandb: 	model_code: VGG11
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch 1/10
40/40 [==============================] - 7s 159ms/step - loss: 2.2994 - accuracy: 0.1079 - val_loss: 2.3032 - val_accuracy: 0.1043
Epoch 2/10
40/40 [==============================] - 6s 144ms/step - loss: 2.3015 - accuracy: 0.1049 - val_loss: 2.2997 - val_accuracy: 0.1018
Epoch 3/10
40/40 [==============================] - 6s 144ms/step - loss: 2.2104 - accuracy: 0.1363 - val_loss: 2.1831 - val_accuracy: 0.1712
Epoch 4/10
40/40 [==============================] - 6s 142ms/step - loss: 1.8745 - accuracy: 0.2512 - val_loss: 1.7623 - val_accuracy: 0.2979
Epoch 5/10
40/40 [==============================] - 6s 144ms/step - loss: 1.6897 - accuracy: 0.3325 - val_loss: 1.6496 - val_accuracy: 0.3610
Epoch 6/10
40/40 [==============================] - 6s 140ms/step - loss: 1.5220 - accuracy: 0.4212 - val_loss: 1.6155 - val_accuracy: 0.4282
Epoch 7/10
40/40 [==============================] - 6s 141ms/step - loss: 1.3886 - accuracy: 0.4900 - val_loss: 1.6146 - val_accuracy: 0.4365
Epoch 

epoch,9
loss,1.04557
accuracy,0.61935
val_loss,1.20569
val_accuracy,0.5685
_runtime,73
_timestamp,1626774674
_step,9
best_val_loss,1.15142
best_epoch,8


epoch,▁▂▃▃▄▅▆▆▇█
loss,██▇▆▅▄▃▃▂▁
accuracy,▁▁▁▃▄▅▆▇▇█
val_loss,██▇▅▄▄▄▂▁▁
val_accuracy,▁▁▂▄▅▆▆▇██
_runtime,▁▂▃▃▄▅▆▇▇█
_timestamp,▁▂▃▃▄▅▆▇▇█
_step,▁▂▃▃▄▅▆▆▇█


...

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: ha041xyk with config:
wandb: 	batch_size: 1024
wandb: 	epochs: 10
wandb: 	hid_dim: 1024
wandb: 	lr: 0.0001
wandb: 	model_code: VGG19
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch 1/10
40/40 [==============================] - 12s 282ms/step - loss: 2.2986 - accuracy: 0.1141 - val_loss: 2.3064 - val_accuracy: 0.0918
Epoch 2/10
40/40 [==============================] - 10s 260ms/step - loss: 2.3026 - accuracy: 0.1032 - val_loss: 2.3005 - val_accuracy: 0.1065
Epoch 3/10
40/40 [==============================] - 10s 262ms/step - loss: 2.1994 - accuracy: 0.1504 - val_loss: 2.0324 - val_accuracy: 0.2007
Epoch 4/10
40/40 [==============================] - 11s 274ms/step - loss: 1.9613 - accuracy: 0.2138 - val_loss: 1.8651 - val_accuracy: 0.2433
Epoch 5/10
40/40 [==============================] - 11s 262ms/step - loss: 1.8281 - accuracy: 0.2549 - val_loss: 1.7857 - val_accuracy: 0.2587
Epoch 6/10
40/40 [==============================] - 11s 274ms/step - loss: 1.7300 - accuracy: 0.2920 - val_loss: 1.6931 - val_accuracy: 0.3067
Epoch 7/10
40/40 [==============================] - 10s 262ms/step - loss: 1.6655 - accuracy: 0.3325 - val_loss: 2.0141 - val_accuracy: 0.2683

epoch,9
loss,1.5799
accuracy,0.37802
val_loss,1.77061
val_accuracy,0.3407
_runtime,157
_timestamp,1626776576
_step,9
best_val_loss,1.67186
best_epoch,8


epoch,▁▂▃▃▄▅▆▆▇█
loss,██▇▅▃▂▂▃▂▁
accuracy,▁▁▂▄▅▆▇▆▇█
val_loss,██▅▃▂▁▅▁▁▂
val_accuracy,▁▁▄▅▆▇▆▇▇█
_runtime,▁▂▃▄▄▅▆▇▇█
_timestamp,▁▂▃▄▄▅▆▇▇█
_step,▁▂▃▃▄▅▆▆▇█


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


* [Experiment1](https://wandb.ai/jeongmin/uncategorized/sweeps/ny3r9m0f?workspace=user-jeongmin)
* [Experiment2](https://wandb.ai/jeongmin/uncategorized/sweeps/maudemkc?workspace=user-jeongmin)

